<a href="https://colab.research.google.com/github/MohsenJadidi/Erdos-Goes-Neural/blob/main/ModifiedCode/Clique_Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
Clique-Train